# Data Preparation

## Packages

In [1]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import  Pool
import time

import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_slope_s(y_axis):
    global data
    x_axis = []
    for i in range(data['sma_len']):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))

    return(slope_tick)

def get_slope_l(y_axis):
    global data
    x_axis = []
    for i in range(data['lma_len']):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

In [3]:
def chunk_ticks(df, number_of_ticks):   
    global data
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df['spread'] = df['Ask'] - df['Bid']
    df = df[['tick', 'spread']]
    
    temp_df = pd.DataFrame()
    tick_avg = []
    spread_avg = []
    tick_sd = []
    
    for i in tqdm(range(0,len(df),number_of_ticks)):
        tick_list = list(df['tick'][i:i+number_of_ticks])
        spread_list = list(df['spread'][i:i+number_of_ticks])
        
        tick_avg.append(np.mean(tick_list))
        spread_avg.append(np.mean(spread_list))
        tick_sd.append(np.std(tick_list))
        
        
    temp_df['tick_avg'] = tick_avg  
    temp_df['spread_avg'] = spread_avg  
    temp_df['tick_sd'] = tick_sd  
    
    return(temp_df)

In [4]:
def before_sma():
    global data    
    data['ssma_list'].append(val)    
    return()

def after_sma():
    global data
    
    data['ssma_list'].popleft()
    data['ssma_list'].append(val)
    data['sema'] = list(pd.DataFrame(list(data['ssma_list'])).ewm(span=data['sma_len']).mean()[0])[data['sma_len'] - 1]
    
    if len(data['sema_ready']) < 2:
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = np.nan

    elif len(data['sema_ready']) > 1:
        data['sema_ready'].popleft()
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = data['sema_ready'][-1] - data['sema_ready'][len(data['sema_ready'])-2]
    
    return()

def before_lma():
    global data    
    data['lsma_list'].append(val)    
    return()

def after_lma():
    global data
    
    data['lsma_list'].popleft()
    data['lsma_list'].append(val)
    data['lema'] = list(pd.DataFrame(list(data['lsma_list'])).ewm(span=data['lma_len']).mean()[0])[data['lma_len'] - 1]
    
    if len(data['lema_ready']) < 2:
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = np.nan

    elif len(data['lema_ready']) > 1:
        data['lema_ready'].popleft()
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = data['lema_ready'][-1] - data['lema_ready'][len(data['lema_ready'])-2]
    
    return()

In [5]:
def roll_sma(ssma_list):
    global data
    sema_val = list(pd.DataFrame(ssma_list).ewm(span=data['sma_len']).mean()[0])[data['sma_len'] - 1]    
    return(sema_val)

def roll_lma(lsma_list):
    global data
    lema_val = list(pd.DataFrame(ssma_list).ewm(span=data['lma_len']).mean()[0])[data['lma_len'] - 1]    
    return(lema_val)

## File paths

In [6]:
year = 2019
source_file_path = f'data/tick_{year}.csv'
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

chunk_file_name = 'chunk_'+file_name
chunk_file_path = os.path.join(path, chunk_file_name)

print(f'source_file_path : {source_file_path}')
print(f'chunk_file_path : {chunk_file_path}')
print(f'target_file_path : {target_file_path}')

source_file_path : data/tick_2019.csv
chunk_file_path : data\chunk_tick_2019.csv
target_file_path : data\tab_tick_2019.csv


## Read data

In [7]:
%%time
#df = pd.read_csv(source_file_path, nrows=100000)
df = pd.read_csv(source_file_path)
df.head()

Wall time: 18 s


,DateTime,Bid,Ask,Volume
0,20190101 22:02:37.254,1.14598,1.14682,4
1,20190101 22:02:38.590,1.14599,1.14682,2
2,20190101 22:02:39.138,1.14599,1.14684,4
3,20190101 22:02:55.787,1.14598,1.14684,4
4,20190101 22:03:02.060,1.14598,1.14684,4


## Data manipulation

In [8]:
%%time
data = {}
data['number_of_ticks'] = 20
df = chunk_ticks(df, data['number_of_ticks'])
df.to_csv(chunk_file_path, index = False)
print(f'Records : {len(df)}')

df = pd.read_csv(chunk_file_path)
df.head()

100%|██████████████████████████████| 1459316/1459316 [06:02<00:00, 4025.07it/s]


Records : 1459316
Wall time: 6min 13s


,tick_avg,spread_avg,tick_sd
0,1.146403,0.000732,0.000036
1,1.146509,0.000655,0.000054
2,1.146389,0.000359,0.000146
3,1.146712,0.000216,0.000038
4,1.146684,0.000273,0.000028


In [9]:
%%time

data['pip_diff'] = 0.00001
data['rsi_window'] = 14
data['rs_max'] = 1e6

data['sma_len'] = 20
data['lma_len'] = 50

data['ssma_list'] = collections.deque([])
data['lsma_list'] = collections.deque([])
data['sema_ready'] = collections.deque([])
data['lema_ready'] = collections.deque([])
df['sema'] = ''
df['lema'] = ''
df['sema_diff'] = ''
df['lema_diff'] = ''


# RSI -----------------------------
df['diff'] = df['tick_avg'].diff()
df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])
df['avg_gain'] = df['gain'].rolling(window=data['rsi_window']).mean()
df['avg_loss'] = df['loss'].rolling(window=data['rsi_window']).mean()
df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= data['rs_max'], data['rs_max']) 
df['rsi'] = 100 - (100 / (df['rs'] + 1))

# Simple Moving Averages ------------------
df['ssma'] = df['tick_avg'].rolling(window=data['sma_len']).mean()
df['ssma_diff'] = df['ssma'].diff()
df['lsma'] = df['tick_avg'].rolling(window=data['lma_len']).mean()
df['lsma_diff'] = df['lsma'].diff()
df['sma_diff'] = df['ssma'] - df['lsma']

df['max_tick'] = df['tick_avg'].rolling(window=data['sma_len']).max()
df['min_tick'] = df['tick_avg'].rolling(window=data['sma_len']).min()

df['max_gap'] = df['max_tick'] -  df['tick_avg']
df['min_gap'] = df['min_tick'] - df['tick_avg']

Wall time: 1.24 s


In [10]:
%%time
# Emas ----------------
df['sema'] = df['tick_avg'].rolling(window=data['sma_len']).progress_apply(roll_sma)
df['lema'] = df['tick_avg'].rolling(window=data['lma_len']).progress_apply(roll_sma)

df['sema_diff'] = df['sema'].diff()
df['lema_diff'] = df['lema'].diff()

1459297it [24:12, 1004.84it/s]
1459267it [24:00, 1012.89it/s]


Wall time: 48min 13s


In [11]:
# Slopes -----------------------------
#df['small_sema_slope'] = df['sema'].rolling(window=data['sma_len']).progress_apply(get_slope_s)
#df['long_sema_slope'] = df['sema'].rolling(window=data['lma_len']).progress_apply(get_slope_l)

In [12]:
%%time

df['ema_diff'] = df['sema'] - df['lema']

# Direction -------------------------
diff_col = 'sema'
df['direction'] = 'same'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] > data['pip_diff']] = 'increase'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] < -data['pip_diff']] = 'decrease'

# Remove NaNs ------------------------
del df['gain']
del df['loss']
df = df.dropna()
df = df.reset_index(drop=True)
print(f'Total records : {len(df)}')

df.tail()

Total records : 1459266
Wall time: 1.14 s


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,ssma_diff,lsma,lsma_diff,sma_diff,max_tick,min_tick,max_gap,min_gap,ema_diff,direction
1459261,1.121196,0.000058,0.000008,1.121331,1.121509,-0.000019,-1.126086e-05,-0.000043,0.000011,0.000026,...,-0.000013,1.121458,-0.000012,-0.000086,1.121493,1.121196,0.000296,0.000000,-0.000177,decrease
1459262,1.121190,0.000044,0.000024,1.121314,1.121502,-0.000017,-6.574689e-06,-0.000006,0.000010,0.000026,...,-0.000013,1.121446,-0.000011,-0.000088,1.121493,1.121190,0.000302,0.000000,-0.000188,decrease
1459263,1.121109,0.000072,0.000014,1.121289,1.121491,-0.000025,-1.170967e-05,-0.000081,0.000008,0.000032,...,-0.000018,1.121434,-0.000012,-0.000093,1.121493,1.121109,0.000384,0.000000,-0.000201,decrease
1459264,1.121161,0.000226,0.000078,1.121273,1.121480,-0.000016,-1.033557e-05,0.000052,0.000011,0.000029,...,-0.000014,1.121424,-0.000011,-0.000097,1.121493,1.121109,0.000331,-0.000052,-0.000207,same
1459265,1.121223,0.000589,0.000045,1.121264,1.121480,-0.000009,1.825067e-07,0.000061,0.000016,0.000020,...,-0.000013,1.121416,-0.000008,-0.000102,1.121457,1.121109,0.000234,-0.000114,-0.000216,same


## Write data to csv

In [13]:
%%time
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

Wall time: 48.2 s


## Print Report

In [14]:
g= df['direction']
print(pd.concat([g.value_counts(), g.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage')))
df.head(5)

          counts  percentage
same      946807   64.882413
decrease  256665   17.588637
increase  255794   17.528949


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,ssma_diff,lsma,lsma_diff,sma_diff,max_tick,min_tick,max_gap,min_gap,ema_diff,direction
0,1.146322,0.000234,0.000011,1.146323,1.146355,1.063391e-06,-3.599718e-07,-0.000009,0.000007,0.000010,...,0.000003,1.146357,-0.000002,-0.000046,1.146361,1.146198,0.000039,-0.000124,-0.000033,same
1,1.146271,0.000306,0.000015,1.146319,1.146357,-3.848168e-06,1.376940e-06,-0.000051,0.000007,0.000011,...,0.000004,1.146353,-0.000005,-0.000037,1.146361,1.146224,0.000090,-0.000047,-0.000038,same
2,1.146260,0.000335,0.000007,1.146314,1.146360,-5.083489e-06,3.357754e-06,-0.000011,0.000007,0.000012,...,0.000002,1.146350,-0.000003,-0.000033,1.146361,1.146228,0.000101,-0.000032,-0.000046,same
3,1.146310,0.000254,0.000028,1.146315,1.146358,8.506514e-07,-1.952792e-06,0.000050,0.000010,0.000012,...,0.000004,1.146342,-0.000008,-0.000021,1.146361,1.146260,0.000052,-0.000050,-0.000043,same
4,1.146294,0.000307,0.000028,1.146312,1.146352,-2.520140e-06,-5.857545e-06,-0.000015,0.000010,0.000009,...,-0.000002,1.146334,-0.000008,-0.000015,1.146361,1.146260,0.000067,-0.000035,-0.000040,same
